In [1]:
import pandas as pd
import numpy as np
import bqplot
import yt

Symbol_T = 'Year'
Symbol_S = 'Total Sightings'
Symbol_D = 'Total Duration'

In [2]:
from bqplot import DateScale, LinearScale, Axis, Lines, Scatter, Bars, Hist, Figure
from bqplot.interacts import (
    FastIntervalSelector, IndexSelector, BrushIntervalSelector,
    BrushSelector, MultiSelector, LassoSelector, PanZoom, HandDraw
)
from traitlets import link

from ipywidgets import ToggleButtons, VBox, HTML, Dropdown, HBox

In [3]:
names = ["date", "city", "state", "country", "shape", "duration_seconds",
         "duration_reported", "description", "report_date", "latitude",
         "longitude"]
fn = "/srv/nbgrader/data/ufo-scrubbed-geocoded-time-standardized.csv"
ufo = pd.read_csv(fn, names = names, parse_dates = ["date", "report_date"])

In [4]:
ufo.head()

,date,city,state,country,shape,duration_seconds,duration_reported,description,report_date,latitude,longitude
0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,2004-04-27,29.883056,-97.941111
1,1949-10-10 21:00:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,-98.581082
2,1955-10-10 17:00:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.200000,-2.916667
3,1956-10-10 21:00:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.978333,-96.645833
4,1960-10-10 20:00:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.418056,-157.803611


In [5]:
ufo['year'] = [i.year for i in ufo['date']]
ufo = ufo.set_index('year')

In [6]:
ufo_temp = ufo.groupby(['state', 'year'])['report_date'].count()
ufo_temp['ab'].loc[1952: 1969]

year
1952    1
1956    1
1960    1
1963    1
1966    2
1967    2
Name: report_date, dtype: int64

In [7]:
ufo_temp_S = ufo.groupby(['state', 'year'])['report_date'].count()
ufo_temp_D = ufo.groupby(['state', 'year'])['duration_seconds'].sum()

In [8]:
# total_sightings = ufo.groupby(['state','year'])['report_date'].count()
# total_time = ufo.groupby(['state','year'])['duration_seconds'].sum()

In [9]:
# total_sightings_year = ufo.groupby('year')
# total_sightings_year = total_sightings_year['date'].count()
# total_duration_year = ufo.groupby('year')['duration_seconds'].sum()

### Define a Figure

In [10]:
total_S = ufo_temp_S['tx']
total_D = ufo_temp_D['tx']

In [11]:
# total_S[:]

In [12]:
#total_S.values()
# total_S.index


In [13]:
import bqplot.pyplot as bqp

In [14]:
dt_x_fast = LinearScale()#min = min(ufo['date']), max = max(ufo['date']))
lin_y = LinearScale()#min = 0, max = 10000)

x_ax = Axis(label = 'Year', scale = dt_x_fast)
y_ay_S = Axis(label = (Symbol_S + ' of the selected state'),
           scale = lin_y, orientation = 'vertical')
y_ay_D = Axis(label = (Symbol_D + ' of the selected state'),
           scale = lin_y, orientation = 'vertical')
# lc = Lines(x = pd.Series(total_S.index.values), y = total_S[:], colors = ['orange'],scales={'x':dt_x_fast,'y':lin_y})
lc = Lines(x = pd.Series(total_S.index.values), 
           y = total_S[:], 
           colors = ['orange'],
           scales={'x':LinearScale(),'y':LinearScale()})

lc2 = Lines(x = pd.Series(total_D.index.values), 
            y = total_D[:],
            colors = ['blue'],
            scales={'x':LinearScale(),'y':LinearScale()})

In [22]:
total_S.index.values[0]

1910

### Define the type of selector we would like

In [16]:
intsel_fast = FastIntervalSelector(scale = dt_x_fast, marks = [lc])

### Define a function that will be called when the FastIntervalSelector is interacted with

In [17]:

def fast_interval_change_callback(change):
    ind = change.new
    print(ind)
#     tot = total_S.loc[int(round(ind[0])):int(ind[1])].count()
    db_fast.value = 'The selected time period is '+ str(change.new)

### Connect the selectors to the function


In [18]:
intsel_fast.observe(fast_interval_change_callback, names = ['selected'])

### Use the HTML widget to see the value of what we are selecting and modify it when an interaction is performed on the selector

In [19]:
db_fast = HTML()

db_fast.value = 'The selected time period is ' + str(intsel_fast.selected)

fig_fast_insel_sights = Figure(marks = [lc], axes = [x_ax, y_ay_S],
                               interaction = intsel_fast)
fig_fast_insel_time = Figure(marks = [lc2], axes = [x_ax, y_ay_D],
                            interaction = intsel_fast)

def watch_dropdown(change):
    db_fast_S.value = str((change.old, change.new))
new_dropdown = Dropdown(options = ['Total Sightings','Total Time'])
new_dropdown.observe(watch_dropdown, names = ['value'])
VBox([db_fast,fig_fast_insel_sights, new_dropdown])


VBox(children=(HTML(value='The selected time period is None'), Figure(axes=[Axis(label='Year', scale=LinearScale(), side='bottom'), Axis(label='Total Sightings of the selected state', orientation='vertical', scale=LinearScale(), side='left')], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, interaction=FastIntervalSelector(marks=[Lines(colors=['orange'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([1910, 1945, 1946, 1947, 1948, 1949, 1950, 1952, 1953, 1954, 1955,
       1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966,
       1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977,
       1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988,
       1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999,
       2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014]), y=array([  1,   1,   1,   1,   1,   3,   1,   4,   4,   6,   3,   2,  12,
         3,   1,   3,   5,   4,   4,   2,   9,   9,   9,  11,   7,   7,
         8,  12,  19,  10,  19,  15,  10,  15,  12,  15,   5,   5,   9,
        13,  15,  13,  13,  12,  11,  14,  12,  17,  15,  11,  38,  39,
        69,  81, 132, 132, 125, 153, 170, 182, 197, 213, 190, 341, 237,
       207, 235, 232, 221,  69]))], scale=LinearScale()), layout=Layout(min_width='125px'), marks=[Lines(colors=['orange'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([1910, 1945, 1946, 1947, 1948, 1949, 1950, 1952, 1953, 1954, 1955,
       1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966,
       1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977,
       1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988,
       1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999,
       2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014]), y=array([  1,   1,   1,   1,   1,   3,   1,   4,   4,   6,   3,   2,  12,
         3,   1,   3,   5,   4,   4,   2,   9,   9,   9,  11,   7,   7,
         8,  12,  19,  10,  19,  15,  10,  15,  12,  15,   5,   5,   9,
        13,  15,  13,  13,  12,  11,  14,  12,  17,  15,  11,  38,  39,
        69,  81, 132, 132, 125, 153, 170, 182, 197, 213, 190, 341, 237,
       207, 235, 232, 221,  69]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0)), Dropdown(options=('Total Sightings', 'Total Time'), value='Total Sightings')))

In [99]:
import us

In [ ]:
cap_select = Dropdown(options=['Total Sightings','Total Duration Time'],description='Caption:')
def onCaptionSelected(change):
    cap = change['new']
    if cap == 'Total Sightings':
        states_map.color = fips_count.to_dict()
    if cap == 'Total Duration Time':
        states_map.color = total_time_all.to_dict()
        
cap_select.observe(onCaptionSelected,'value') 

In [ ]:
abbr_to_fits = us.states.mapping('abbr', 'fips')
ufo["fips"] = ufo["state"].apply(lambda a: int(abbr_to_fits.get(str(a).upper(), -1)))
fips_count = ufo.groupby("fips")["duration_seconds"].count()
total_time_all = ufo.groupby("fips")["duration_seconds"].sum()

map_styles = {'scales': {'projection': bqplot.AlbersUSA(),
                         'color': bqplot.ColorScale(colors=["red","yellow", "blue"])},
              'color': fips_count.to_dict()}
map_tt = bqplot.Tooltip(fields = ['name','color'], labels = ['state','duration_seconds'])
states_map = bqplot.Map(map_data=bqplot.topo_load('map_data/USStatesMap.json'),
                        interactions = {'click':'select','hover':'tooltip'},
                        unslected_styles = {'opacity': 0.4},
                        **map_styles, tooltip = map_tt, display_legend = False, labels=['UFO Sightings'])
fig_map = bqplot.Figure(marks=[states_map], fig_margins = {60,60,40,40},
                    title='UFO Sightings in the United States')

In [ ]:
def observe_selected(change):
    print(states_map.selected[-1])
    for abbr,fips in abbr_to_fits.items():
        if fips == str(states_map.selected[-1]):
            state_selected = abbr.lower()
            if cap_select.value == 'Total Sightings':
                plot_all.x = total_sightings[state_selected].index.values
                plot_all.y = total_sightings[state_selected][:]
    print(states_map.selected)
states_map.observe(observe_selected, 'selected')

In [ ]:
plot_all = Lines(x=total_sightings_year.index.values,y=total_sightings_year[:],colors = ['orange'],scales={'x':dt_x_fast,'y':lin_y})
fig_all = Figure(marks=[plot_all],axes = [x_ax, y_ay_S],)

In [ ]:
total_sightings_year.index.values

In [ ]:
HBox([VBox([cap_select,fig_map]),fig_all])

In [ ]:
len(ufo)